# Run Tensorflow code as SageMaker Training Job

This code shows how to run code written in notebook as SageMaker Training Job by using sagemill <br>
In this example, we use `sagemaker.tensorflow.Tensorflow` as Estimator

#### assumptions
- this notebook is from "conda_python3" in SageMaker notebook instance

In [ ]:
!conda install -y tensorflow==2.1.0 pandas scikit-learn

## Parameters
The cell with "parameters" tag is used as hyperparameters in SageMaker Training Job. <br>
So these can be overwritten by `hyperparameters` in `sagemaker.tensorflow.Tensorflow`

#### How to add tags
1. Click 'View' tab on the notebook
2. Click 'Cell Toolbar'
3. Click 'Tags'
4. Input tag name to the cell
5. Click "Add tag"

In [ ]:
epochs = 1
batch_size = 8
data_dir = './dataset'
local_model_dir = './model'
# for iris
num_class = 3
dim_data = 4

In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras import Model

x_filename = 'x.npy'
y_filename = 'y.npy'

## Download dataset & upload it to S3
The cell with "sagemaker" tag is ignored when generating python script from the notebook by `Converter.generate_pyfile`

In [ ]:
import sagemaker
from sklearn import datasets
from sklearn.model_selection import train_test_split
import shutil

iris = datasets.load_iris()

# In local, save only sample for debugging
os.makedirs(data_dir, exist_ok=True)
x_train, _, y_train, _ = train_test_split(iris.data, iris.target, train_size=100)
np.save(os.path.join(data_dir, x_filename), x_train)
np.save(os.path.join(data_dir, y_filename), y_train)

# In S3, upload all data for full training
tmp_dir = 'tmp_dataset'
os.makedirs(tmp_dir, exist_ok=True)
np.save(os.path.join(tmp_dir, x_filename), iris.data)
np.save(os.path.join(tmp_dir, y_filename), iris.target)
s3_input = sagemaker.Session().upload_data(path=tmp_dir, key_prefix='datasets/sagemill_tf')
shutil.rmtree(tmp_dir)

## Code to train your model
You can write any code here to train your Tensorflow model

In [ ]:
# define Tensorflow model
def create_model(dim_data: int, num_class: int) -> Model:
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(dim_data,)),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(num_class, activation='softmax')
    ])
  
    model.compile(optimizer='adam', 
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy']) 
    return model

model = create_model(dim_data, num_class)

In [ ]:
# training code
x_train = np.load(os.path.join(data_dir, x_filename))
y_train = np.load(os.path.join(data_dir, y_filename))

model.fit(x_train, y_train)
os.makedirs(local_model_dir, exist_ok=True)
model.save(os.path.join(local_model_dir, 'my_model.h5'))

## Run SageMaker Training Job

- `Converter.generate_pyfile` generates `entrypoint.py` from this notebook.
- Set `entrypoint.py` as `entry_point` of `samaker.tensorflow.Tensorflow`
- `hyper_params` corresponds with the cell tagged "parameters" <br>
  In the training job, the parameters are overwritten to the values of `hyper_params`
- **save this notebook file before runnning the following cell**

In [ ]:
from sagemill import Converter
from sagemaker.tensorflow import TensorFlow

role = sagemaker.get_execution_role()
max_run_time = 24 * 60 * 60 * 1  # 1 day
entry_point = 'entrypoint.py'
hyper_params = {
    'batch_size': 64,
    'epochs': 10,
    'local_model_dir': '/opt/ml/model',
    'data_dir': '/opt/ml/input/data/training',
}

# Convert this notebook to Python file
Converter.generate_pyfile('./train_tf.ipynb', entry_point)


estimator = TensorFlow(
    entry_point=entry_point,
    role=role,
    base_job_name='sagemill-tf',
    train_instance_count=1,
    framework_version="2.1.0",
    py_version="py3",
    train_instance_type='ml.c5.xlarge',
    train_max_run=max_run_time,
    script_mode=True,
    hyperparameters=hyper_params)

estimator.fit(inputs=s3_input)